In [ ]:
import numpy as np
import mne
from tqdm import tqdm
import os
mne.set_log_level('WARNING')
from mne.viz import plot_topomap as plot_topo_ZKP
import matplotlib.pyplot as plt
import matplotlib as mpl


In [ ]:
s1_epochs_before=mne.read_epochs(os.path.join('E:\ZKP\drug persons\eeg\export\EpochData', "MBT_S1-epo.fif"))
s2_epochs_before=mne.read_epochs(os.path.join('E:\ZKP\drug persons\eeg\export\EpochData', "MBT_S2-epo.fif"))
s1_epochs_after=mne.read_epochs(os.path.join('E:\ZKP\drug persons\eeg\export\EpochData', "MAT_S1-epo.fif"))
s2_epochs_after=mne.read_epochs(os.path.join('E:\ZKP\drug persons\eeg\export\EpochData', "MAT_S2-epo.fif"))
s1_epochs_Health=mne.read_epochs(os.path.join('E:\ZKP\drug persons\eeg\export\EpochData', "HC_S1-epo.fif"))
s2_epochs_Health=mne.read_epochs(os.path.join('E:\ZKP\drug persons\eeg\export\EpochData', "HC_S2-epo.fif"))
s1_epochs_HealthA=mne.read_epochs(os.path.join('E:\ZKP\drug persons\eeg\export\EpochData', "HCA_S1-epo.fif"))
s2_epochs_HealthA=mne.read_epochs(os.path.join('E:\ZKP\drug persons\eeg\export\EpochData', "HCA_S2-epo.fif"))

Processing the data

In [ ]:
s1_spectrum=s1_epochs_before.compute_psd(picks="eeg", fmin=1, fmax=44,method='multitaper',n_jobs=8)
s1_spectrum_after=s1_epochs_after.compute_psd(fmin=1, fmax=44,method='multitaper',n_jobs=8)
s1_spectrum_Health=s1_epochs_Health.compute_psd(fmin=1, fmax=44,method='multitaper',n_jobs=8)


s2_spectrum=s2_epochs_before.compute_psd(picks="eeg", fmin=1, fmax=44,method='multitaper',n_jobs=8)
s2_spectrum_after=s2_epochs_after.compute_psd(fmin=1, fmax=44,method='multitaper',n_jobs=8)
s2_spectrum_Health=s2_epochs_Health.compute_psd(fmin=1, fmax=44,method='multitaper',n_jobs=8)


Define the function to plot the topomap

In [ ]:
vmax_g,vmin_g=None,None
def plot_spec_topo(pectrums_,band=None,vlim=None):
    global vmax_g,vmin_g
    bands = {'gamma': (30, 44),'beta': (13, 30),'alpha': (8, 13),'theta': (4, 8),'delta':(1, 4)}
    def deal_psd(spectrum_):
        psd=spectrum_.get_data()
        psd*=1e6*1e6
        psd /= np.sum(psd, axis=-1, keepdims=True)
        psd*=100
        mask=(spectrum_.freqs >= bands[band][0]) & (spectrum_.freqs < bands[band][1])
        psd=psd[:,:,mask]
        return psd.mean(axis=-1)

    psd_MBT=deal_psd(pectrums_[0])
    psd_MAT=deal_psd(pectrums_[1])
    psd_HC=deal_psd(pectrums_[2])
    psd_HCA=deal_psd(pectrums_[3])

    psd_MBT_b=psd_MBT.mean(axis=-1)
    psd_MAT_b=psd_MAT.mean(axis=-1)
    psd_HC_b=psd_HC.mean(axis=-1)
    psd_HCA_b=psd_HCA.mean(axis=-1)

    psd_MBT_b_mean,psd_MBT_b_std=psd_MBT_b.mean(),psd_MBT_b.std()
    psd_MAT_b_mean,psd_MAT_b_std=psd_MAT_b.mean(),psd_MAT_b.std()
    psd_HC_b_mean,psd_HC_b_std=psd_HC_b.mean(),psd_HC_b.std()
    psd_HCA_b_mean,psd_HCA_b_std=psd_HCA_b.mean(),psd_HCA_b.std()

    print(psd_MBT_b_mean.round(3),psd_MAT_b_mean.round(3),psd_HC_b_mean.round(3),psd_HCA_b_mean.round(3))
    print(psd_MBT_b_std.round(3),psd_MAT_b_std.round(3),psd_HC_b_std.round(3),psd_HCA_b_std.round(3))


    psd_MBT_a=psd_MBT.mean(axis=0)
    psd_MAT_a=psd_MAT.mean(axis=0)
    psd_HC_a=psd_HC.mean(axis=0)
    psd_HCA_a=psd_HCA.mean(axis=0)


    if band in ['alpha','theta']:
        vmax=psd_HC_a.mean()+2.5*psd_HC_a.std()
        vmin=psd_HC_a.mean()-1.8*psd_HC_a.std()
    else:
        vmax=psd_MBT_a.mean()+1.8*psd_MBT_a.std()
        vmin=psd_MBT_a.mean()-1.8*psd_MBT_a.std()

    if vlim is not None:
        vmax=vlim[1]
        vmin=vlim[0]
    vmax_g,vmin_g=vmax,vmin




    fig, axes = plt.subplots(1, 1, figsize=(3, 3))
    
    # -------------un-comment the line below to plot the topomap----------
    # plot_topo_ZKP(psd_MBT_a, pectrums_[0].info, size=5,res=800,axes=axes,vlim=(vmin,vmax))
    # plot_topo_ZKP(psd_MAT_a, pectrums_[0].info, size=5,res=800,axes=axes,vlim=(vmin,vmax))
    # plot_topo_ZKP(psd_HC_a, pectrums_[0].info, size=5,res=800,axes=axes,vlim=(vmin,vmax))



In [ ]:

# plot_spec_topo([s1_spectrum,s1_spectrum_after,s1_spectrum_Health,s1_spectrum_Health_A],'delta',vlim=[1.55,3.96])
# plot_spec_topo([s2_spectrum,s2_spectrum_after,s2_spectrum_Health,s2_spectrum_Health_A],'delta',vlim=[0.05,0.13])


Plot the colorbar

In [ ]:
from matplotlib.ticker import StrMethodFormatter
import copy
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 1, figsize=(0.5, 4))
fig.subplots_adjust(hspace=4)

# 第一个colorbar使用线性的Normalize.
cmap1 = copy.copy(mpl.cm.Reds)
norm1 = mpl.colors.Normalize(vmin=vmin_g, vmax=vmax_g)
im1 = mpl.cm.ScalarMappable(norm=norm1, cmap=cmap1)
cbar1 = fig.colorbar(
    im1, cax=axes, orientation='vertical',
    ticks=np.linspace(vmin_g, vmax_g, 5),
    label='',
)
formatter = StrMethodFormatter('{x:.2f}')
cbar1.ax.yaxis.set_major_formatter(formatter)

new_size = 30
cbar1.ax.yaxis.set_tick_params(labelsize=new_size)


plt.show()
